# Setup


In [1]:
# %%capture
# %pip install langchain==0.3.27
# %pip install jupyterlab==4.4.9

In [22]:
import os
import time
import uuid
from dotenv import load_dotenv

import pandas as pd

__import__("pysqlite3")
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.tools import tool, create_retriever_tool
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

In [3]:
load_dotenv()

GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

In [4]:
# MODELS
model_gemini_2_0_flash = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0,
    max_tokens=100,
    max_retries=1,
)

# model_embedding_001 = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001", google_api_key=GOOGLE_API_KEY
# )

model_embedding_001 = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2", 
    model_kwargs={"device": "cpu"}
)

E0000 00:00:1759253712.419724  100454 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
/tmp/ipykernel_100454/2822504156.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_embedding_001 = HuggingFaceEmbeddings(


# ReAct Agent Example


## Create agent


In [5]:
@tool
def find_sum(a: int, b: int) -> int:
    """
    Add two integers and return their sum.
    """
    return a + b


@tool
def find_product(a: int, b: int) -> int:
    """
    Multiply two integers and return their product.
    """
    return a * b

In [6]:
agent_tools = [find_sum, find_product]

system_prompt = SystemMessage(
    """
    You are a mathematician who solves math problems using tools only. 
    Do not solve any problem yourself — always use the available tools to find the solution.
    """
)

agent_graph = create_react_agent(
    model=model_gemini_2_0_flash, prompt=system_prompt, tools=agent_tools
)

## Invoke agent


In [7]:
# # Example 1
# inputs = {"messages": [("user", "what is the sum of 2 and 3 ?")]}
# result = agent_graph.invoke(inputs)

# print(f"Agent returned : {result['messages'][-1].content} \n")
# print("Step by Step execution : ")
# for message in result["messages"]:
#     print(message.pretty_repr())

In [8]:
# # Example 2
# inputs = {"messages": [("user", "What is 3 multipled by 2 and 5 + 1 ?")]}
# result = agent_graph.invoke(inputs)

# print(f"Agent returned : {result['messages'][-1].content} \n")
# print("Step by Step execution : ")
# for message in result["messages"]:
#     print(message.pretty_repr())

In [9]:
# # Example 3
# inputs = {"messages": [("user", "what is the sum of 2.1 and 3.7 ?")]}
# result = agent_graph.invoke(inputs)

# print(f"Agent returned : {result['messages'][-1].content} \n")
# print("Step by Step execution : ")
# for message in result["messages"]:
#     print(message.pretty_repr())

## Debug agent


In [10]:
agent_graph_debug = create_react_agent(
    model=model_gemini_2_0_flash, prompt=system_prompt, tools=agent_tools, debug=True
)

In [11]:
# inputs = {"messages": [("user", "what is the sum of 2 and 3 ?")]}
# result = agent_graph_debug.invoke(inputs)

# Product Q&A Chatbot


## Data


In [12]:
product_price_df = pd.read_csv("data/smartphone_prices.csv")
product_price_df

,Name,Price,ShippingDays
0,Zenith One,899,5
1,AeroPhone Lite,299,3
2,Photon X Ultra,1199,7
3,Nimbus Mini,749,4
4,Titan Max 5G,999,6


In [13]:
loader = PyPDFLoader("./data/smartphone_descriptions.pdf")
docs = loader.load()
docs

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-30T17:20:58+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-30T17:20:58+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': './data/smartphone_descriptions.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Zenith One\nThe Zenith One packs flagship performance into a slim design. It features the Snapdragon X3 chip,\n12GB of RAM, and 256GB of storage. Its 6.7-inch OLED display with 120Hz refresh rate delivers\nsmooth visuals, while the triple camera system excels in low-light photography.\n AeroPhone Lite\nAeroPhone Lite is designed for budget-conscious users who don’t want to compromise on quality.\nWith a MediaTek Helio G95 processor, 6GB RAM, and 128GB storage, it offers snappy\nperformance for everyday tasks. Its 5000mAh battery ensures two full days of usage.\n Photon X Ultra\n

## Tools


In [14]:
@tool
def get_smartphone_price(smartphone_name: str) -> int:
    """
    Returns the price of a smartphone by name (case-insensitive substring match).
    If no match is found, returns -1.
    """
    pattern = f"^{smartphone_name.strip()}"
    matches = product_price_df[
        product_price_df["Name"].str.contains(pattern, case=False, na=False)
    ]
    if matches.empty:
        return -1
    return int(matches["Price"].iloc[0])


print(get_smartphone_price.invoke("zenith"))
print(get_smartphone_price.invoke("asdf"))

899
-1


In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

feature_store = Chroma.from_documents(
    documents=splits, embedding=model_embedding_001
)

get_product_features = create_retriever_tool(
    feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about smartphones. It lists the available smartphones
    and their features including camera, memory, storage, design and advantages
    """
)

feature_store.as_retriever().invoke("Tell me about the Zenith One")

[Document(id='99108aca-9893-457a-85c5-873e9ad575da', metadata={'keywords': '', 'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'author': '(anonymous)', 'page': 0, 'source': './data/smartphone_descriptions.pdf', 'title': '(anonymous)', 'subject': '(unspecified)', 'trapped': '/False', 'total_pages': 1, 'moddate': '2025-09-30T17:20:58+00:00', 'page_label': '1', 'creationdate': '2025-09-30T17:20:58+00:00'}, page_content='Zenith One\nThe Zenith One packs flagship performance into a slim design. It features the Snapdragon X3 chip,\n12GB of RAM, and 256GB of storage. Its 6.7-inch OLED display with 120Hz refresh rate delivers\nsmooth visuals, while the triple camera system excels in low-light photography.\n AeroPhone Lite\nAeroPhone Lite is designed for budget-conscious users who don’t want to compromise on quality.\nWith a MediaTek Helio G95 processor, 6GB RAM, and 128GB storage, it offers snappy\nperformance for everyday tasks. Its 5000mAh battery ensures

## Chatbot

In [16]:
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about smartphones sold by your company.
    To answer questions about smartphones, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)
checkpointer = MemorySaver() # conversation memory

product_QnA_agent=create_react_agent(
    model=model_gemini_2_0_flash,
    tools=[get_smartphone_price, get_product_features],
    prompt=system_prompt,
    debug=False,
    checkpointer=checkpointer
)

In [17]:
# To maintain memory, each request should be in the context of a thread.
# Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

inputs = {"messages": [HumanMessage("What are the features and pricing for Zenith One?")]}

# Use streaming to print responses as the agent  does the work.
# This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message = stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for Zenith One?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (f65da3ad-ede7-44bc-82fe-24de822602ee)
 Call ID: f65da3ad-ede7-44bc-82fe-24de822602ee
  Args:
    query: Zenith One
  get_smartphone_price (75e4d671-4310-40d3-b7f4-a03b15a2f213)
 Call ID: 75e4d671-4310-40d3-b7f4-a03b15a2f213
  Args:
    smartphone_name: Zenith One
================================= Tool Message =================================
Name: get_smartphone_price

899
================================== Ai Message ==================================

The Zenith One features a Snapdragon X3 chip, 12GB of RAM, and 256GB of storage. It has a 6.7-inch OLED display with a 120Hz refresh rate and a triple camera system. The price of the Zenith One is $899.


## Execute

In [23]:
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a smartphone",
    "Give me a list of available smartphone names",
    "Tell me about the features of Zenith One",
    "How much does it cost?",
    "Give me similar information about TitanMax",
    "What info do you have on Nimbus ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    time.sleep(1)
    print(f"----------------------------------------\nUSER : {input}")
    user_message = {"messages":[HumanMessage(input)]}
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    print(f"AGENT : {ai_response['messages'][-1].content}")

----------------------------------------
USER : Hello
AGENT : Hello! How can I help you today?
----------------------------------------
USER : I am looking to buy a smartphone
AGENT : Great! I can help you with that. What are you looking for in a smartphone? Do you have any specific features or brands in mind?
----------------------------------------
USER : Give me a list of available smartphone names
AGENT : The available smartphones are Nimbus Mini and Titan Max 5G.
----------------------------------------
USER : Tell me about the features of Zenith One
AGENT : The Zenith One packs flagship performance into a slim design. It features the Snapdragon X3 chip, 12GB of RAM, and 256GB of storage. Its 6.7-inch OLED display with 120Hz refresh rate delivers smooth visuals, while the triple camera system excels in low-light photography.
----------------------------------------
USER : How much does it cost?
AGENT : The Zenith One costs $899.
----------------------------------------
USER : Give

In [20]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  Zenith")
execute_prompt("USER 2", config_2, "Tell me about the features of Nimbus Mini")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")


USER 1: The Zenith One packs flagship performance into a slim design. It features the Snapdragon X3 chip, 12GB of RAM, and 256GB of storage. Its 6.7-inch OLED display with 120Hz refresh rate delivers smooth visuals, while the triple camera system excels in low-light photography.

USER 2: The Nimbus Mini is a compact 5.8-inch smartphone with the A14 Fusion chip, 8GB RAM, and 256GB storage. It is ideal for users who prefer smaller phones with high performance.

USER 1: The price of Zenith One is 899.

USER 2: The price of the Nimbus Mini is $749.


# Orders Chatbot